In [1]:
from web3 import Web3
import rlp
from ethereum.transactions import Transaction
from solc import compile_files
from json import dumps, loads

In [2]:
web3 = Web3(Web3.HTTPProvider("https://rinkeby.infura.io/MSNyVuE56cCNMxT1C4BA"))

In [3]:
account = web3.eth.account.create('password')
print(account.address)
print(account.privateKey.hex())

0xaaD64BEf1b6Fa3646859d10e6EE670D01811b908
0x88b7d17e7e9cb668ec1700c3b3a7efd5229ec2bf8e165be948f514412857c6e9


In [3]:
account_addr = '0xaaD64BEf1b6Fa3646859d10e6EE670D01811b908'
account_pk = '0x88b7d17e7e9cb668ec1700c3b3a7efd5229ec2bf8e165be948f514412857c6e9'

In [4]:
res = compile_files(['contracts/4tests.sol'], optimize=True, optimize_runs=500)

controller = res['contracts/4tests.sol:ICO_controller']
crowdsale = res['contracts/4tests.sol:WhitelistedCrowdsale']
token = res['contracts/4tests.sol:MFC_Token']

f = open('controller_abi.txt', 'w')
f.write(dumps(controller['abi']))
f.close()

In [7]:
res2 = compile_files(['contracts/4tests_Holder.sol'], optimize=True, optimize_runs=100)

holder = res2['contracts/4tests_Holder.sol:Holder']

f = open('holder_abi.txt', 'w')
f.write(dumps(holder['abi']))
f.close()

In [8]:
f = open('controller_bin.txt', 'w')
f.write(dumps(controller['bin']))
f.close()

In [9]:
holder = web3.eth.contract(abi=holder['abi'], bytecode=holder['bin'])
tx_h = Transaction(
    nonce=web3.eth.getTransactionCount(account_addr),
    to=b'',
    gasprice=web3.eth.gasPrice,
    startgas=5000000,
    value=0,
    data=holder.bytecode
    )
tx_h.sign(account_pk)
raw_tx_h = rlp.encode(tx_h)
raw_tx_h_hex = web3.toHex(raw_tx_h)
tx_h_hash = web3.eth.sendRawTransaction(raw_tx_h_hex)
print(tx_h_hash.hex())

0x5115d9666b267a2677581ed7f7ac711e99d79304a6b0853d55327a7caa7d49c1


In [11]:
tx_h_receipt = web3.eth.getTransactionReceipt(tx_h_hash)
holder_address = tx_h_receipt['contractAddress']
print(holder_address)

0x7eD566ab6C3AAd0234a4CB694B5a487f5adFe2fE


In [17]:
kwargs = {
    '_holder': '0x7eD566ab6C3AAd0234a4CB694B5a487f5adFe2fE',
    '_incentive_program': account_addr
}
args = ('0x7eD566ab6C3AAd0234a4CB694B5a487f5adFe2fE', account_addr)
contract = web3.eth.contract(abi=controller['abi'], bytecode=controller['bin'])
txo = {'from': account_addr}
print(dir(contract))
tx = contract.constructor(*args).buildTransacyion(txo)
tx['nonce'] = web3.eth.getTransactionCount(account_addr)
signed = web3.eth.account.signTransaction(tx, account_pk)
tx_hash = web3.eth.sendRawTransaction(raw_tx_hex)
print(tx_hash.hex())

tx_receipt = web3.eth.getTransactionReceipt(tx_hash)
print(tx_receipt)

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_encode_constructor_data', '_find_matching_event_abi', '_find_matching_fn_abi', '_prepare_transaction', '_return_data_normalizers', 'abi', 'address', 'asm', 'ast', 'buildTransaction', 'bytecode', 'bytecode_runtime', 'call', 'clone_bin', 'deploy', 'dev_doc', 'encodeABI', 'estimateGas', 'eventFilter', 'events', 'factory', 'functions', 'interface', 'metadata', 'opcodes', 'src_map', 'src_map_runtime', 'transact', 'user_doc', 'web3']


AttributeError: type object 'Contract' has no attribute 'constructor'

In [36]:
tx_receipt = web3.eth.getTransactionReceipt(tx_hash)
print(tx_receipt)

AttributeDict({'from': '0xaad64bef1b6fa3646859d10e6ee670d01811b908', 'transactionHash': HexBytes('0x2441dc5cd848f3d9121315ef62cf7333a564637bbd44674628ecf2a522fff3e0'), 'blockNumber': 1718227, 'blockHash': HexBytes('0xdbadc7818d73ddf93f57fb1a664355b567c50930cb30685a56a97619ad74980e'), 'logsBloom': HexBytes('0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000'), 'to': None, 'status': 1, 'cumulativeGasUsed': 4643488, 'transactionIndex': 2, 'contractAddress': '0x4c71cF081502b0DaD868d1BA1Fd8f703b2cFFd6e', 'gasUsed': 4588601, 'logs': []}